In [1]:
! pip install PyTDC

In [2]:
import time

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim

from tdc.multi_pred import PPI

In [3]:
ts = time.time()
generator = torch.Generator().manual_seed(12)

In [4]:
data = PPI(name = 'HuRI')
data = data.neg_sample(frac = 1)
split = data.get_split(frac=[0.8, 0, 0.2], seed=12)
train_full = split['train']
valid_full = split['valid']
test_full = split['test']

Found local copy...
Loading...
Done!


In [5]:
amino_acids = 'ARNDCEQGHILKMFPSTWYVX'
hydro = [1.8, -4.5, -3.5, -3.5, 2.5, -3.5, -3.5, -0.4, -3.2, 4.5, 3.8, -3.9,
         1.9, 2.8, -1.6, -0.8, -0.7, -0.9, -1.3, 4.2, -0.49]
hydro = (np.array(hydro) - np.mean(hydro)) / np.std(hydro)
mass = [89.1, 174.2, 132.12, 133.11, 121.16, 147.13, 146.15,
        75.07, 155.16, 131.18, 131.18, 146.19, 149.21, 165.19,
        115.13, 105.09, 119.12, 204.23, 181.19, 117.15, 136.9]
mass = (np.array(mass) - np.mean(mass)) / np.std(mass)
volume = [88.6, 173.4, 114.1, 111.1, 108.5, 138.4, 143.8,
          60.1, 153.2, 166.7, 166.7, 168.6, 162.9, 189.9,
          112.7, 89, 116.1, 227.8, 193.6, 140, 141.3]
volume = (np.array(volume) - np.mean(volume)) / np.std(volume)
pka = [2.34, 2.17, 2.02, 1.88, 1.96, 2.19, 2.17,
       2.34, 1.82, 2.36, 2.36, 2.18, 2.28, 1.83,
       1.99, 2.21, 2.09, 2.83, 2.20, 2.32, 2.18]
pka = (np.array(pka) - np.mean(pka)) / np.std(pka)
pkb = [9.69, 9.04, 8.8, 9.6, 10.28, 9.67, 9.13, 9.6, 9.17, 9.6, 9.6, 
       8.95, 9.21, 9.13, 10.6, 9.15, 9.1, 9.39, 9.11, 9.62, 9.42]
pkb = (np.array(pkb) - np.mean(pkb)) / np.std(pkb)
pki = [6, 10.76, 5.41, 2.77, 5.07, 3.22, 5.65, 5.97, 7.59, 6.02, 5.98,
       9.74, 5.74, 5.48, 6.3, 5.68, 5.6, 5.89, 5.66, 5.96, 6.02]
pki = (np.array(pki) - np.mean(pki)) / np.std(pki)
charge = np.array([0,1,0,-1,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,0,0,0])
polar = np.array([0,1,1,1,0,1,1,0,1,0,0,1,0,0,0,1,1,0,1,0,0.5])
Hdonor = np.array([0,-1,0,1,0,0,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0])
freq = np.array([0.0777, 0.0627, 0.0336, 0.0524, 0.0078, 0.0315, 0.0859,
                 0.073, 0.0192, 0.0666, 0.0891, 0.0776, 0.0241, 0.0361, 
                 0.0435, 0.0466, 0.0487, 0.0102, 0.03, 0.0817, 0.05])
Pconserved = np.array([0.2063, 0.25, 0.1733, 0.262, 0.4095, 0.2244, 0.2960,
                 0.3848, 0.1874, 0.2212, 0.397, 0.2986, 0.1772, 0.3914,
                 0.3373, 0.1689, 0.1771, 0.6068, 0.4085, 0.2338, 0.2906])

aadata = np.stack([hydro, mass, volume, pka, pkb, pki, charge,
                               polar, Hdonor, freq, Pconserved], axis=1)

aatoidx = {}
for i, amino_acid in enumerate(amino_acids):
    aatoidx[amino_acid] = i

In [6]:
# min_len = 8297
# max_len = 33472

# Protein1 = train_full.Protein1
# Protein2 = train_full.Protein2
# Label = train_full.Y
# p1 = []
# p2 = []
# labs = []
# for idx in train_full.index:
#     prot1 = Protein1.loc[idx].replace('*','')
#     prot2 = Protein2.loc[idx].replace('*','')
#     y = Label.loc[idx]
#     nprot1 = []
#     nprot2 = []
#     add = True
#     if len(prot1) <= max_len and len(prot2) <= max_len:
#         if len(prot1) >= min_len and len(prot2) >= min_len:
#             for i in range(max_len):
#                 if i < len(prot1):
#                     if prot1[i] in amino_acids:
#                         nprot1.append(np.concatenate((
#                                 aadata[aatoidx[prot1[i]],:], 
#                                 np.array([float(i)/len(prot1)]))))
#                     else:
#                         add = False
#                         break
#                 else:
#                     nprot1.append(np.array([0,0,0,0,0,0,0,0,0,0,0,0]))
#                 if i < len(prot2):
#                     if prot2[i] in amino_acids:
#                         nprot2.append(np.concatenate((
#                                 aadata[aatoidx[prot2[i]],:], 
#                                 np.array([float(i)/len(prot2)]))))
#                     else:
#                         add = False
#                         break
#                 else:
#                     nprot2.append(np.array([0,0,0,0,0,0,0,0,0,0,0,0]))
#             if add:
#                 p1.append(torch.tensor(nprot1).type(torch.long))
#                 p2.append(torch.tensor(nprot2).type(torch.long))
#                 labs.append(y)

# p1 = torch.stack(p1).type(torch.float)
# p2 = torch.stack(p2).type(torch.float)
# labs = torch.tensor(labs).type(torch.float)

In [7]:
max_len = 3268
p1 = torch.load('p1_1663_3268.pt')
p2 = torch.load('p2_1663_3268.pt')
labs = torch.load('labs_1663_3268.pt')

In [8]:
p1.shape, p2.shape, labs.shape

(torch.Size([2597, 3268, 12]),
 torch.Size([2597, 3268, 12]),
 torch.Size([2597]))

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Running with {device}')

Running with cuda


In [10]:
class HuRI2d(Dataset):
    def __init__(self, labs, p1, p2, loss_fn):
        # labs is an (n_samples,)-long torch tensor of 0/1 interaction scores
        # prot1 is an (n_samples, max_len, n_feats)-shaped tensor of protein 1 data
        # prot2 is an (n_samples, max_len, n_feats)-shaped tensor of protein 2 data
        # loss_fn is one of 'BCE', 'Cross', 'KLDiv', 'SoftMargin', or 'MSE'
        self.labs = labs
        self.p1 = p1
        self.p2 = p2
    
    def __len__(self):
        return self.labs.shape[0]
    
    def __getitem__(self, idx):
        label = torch.zeros(2)
        label[self.labs[idx].type(torch.long)] = 1
        if loss_fn == 'BCE' or loss_fn == 'Cross':
            pass
        elif loss_fn == 'SoftMargin' or loss_fn == 'MSE' or loss_fn == 'KLDiv':
            for i in range(2):
                if label[i] == 0:
                    label[i] = -1
        return (self.p1[idx,:,:], self.p2[idx,:,:]), label.type(torch.float)

In [11]:
class ConvolutionalClassifier2d(nn.Module):
    def __init__(self, c, k, n):
        super().__init__()
        self.c = c
        self.k = k
        self.n = n
        self.InputConv = nn.Conv2d(2, self.c, (self.k, 12), 
                                   padding='same').to(device)
        self.BasicConvs = []
        for i in range(2 * self.n):
            self.BasicConvs.append(nn.Conv2d(int(self.c/2), self.c, (self.k,1),
                                             padding='same').to(device))
        self.Plain = nn.Conv2d(int(self.c/2), self.c, (self.k, 1),
                               padding='same').to(device)
        self.EndConv1 = nn.Conv2d(int(self.c/2), self.c, (1,1)).to(device)
        self.EndConv2 = nn.Conv2d(self.c, 2, (1,1)).to(device)
        self.Lin1 = nn.Linear(max_len*12*2, 64).to(device)
        self.Lin2 = nn.Linear(64, 2).to(device)
    
    def forward(self, p1, p2):
        x = self.InputConv(torch.stack((p1, p2), dim=1))
        for i in range(self.n):
            x = self.BasicConvs[i](F.glu(F.layer_norm(x, x.shape[1:]), dim=1)) + x
        x = self.Plain(F.glu(F.layer_norm(x, x.shape[1:]), dim=1))
        for i in range(self.n, 2*self.n):
            x = self.BasicConvs[i](F.glu(F.layer_norm(x, x.shape[1:]), dim=1)) + x
        x = F.glu(F.layer_norm(x, x.shape[1:]), dim=1)
        x = F.dropout(self.EndConv1(x), p=0.5)
        x = self.EndConv2(x).flatten(start_dim=1)
        x = F.relu(self.Lin1(F.dropout(x, p=0.5)))
        x = self.Lin2(x)
        return F.softmax(x, dim=1)

In [12]:
# loss_fn_name is one of 'BCE', 'Cross', 'KLDiv', 'SoftMargin', or 'MSE'
loss_fn_name = 'MSE'
loss_fn = nn.MSELoss(reduction='sum')

batch_size = 4

c = 256
k = 5
n = 10
model = ConvolutionalClassifier2d(c, k, n)

learning_rate = 0.01

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate)


dataset = HuRI2d(labs, p1, p2, loss_fn_name)

train_test = random_split(dataset, [int(labs.shape[0]*0.8), 
                                    labs.shape[0]-int(labs.shape[0]*0.8)],
                          generator=generator)


train_loader = DataLoader(train_test[0], batch_size=batch_size, shuffle=True)
test_loader = DataLoader(train_test[1], batch_size=batch_size, shuffle=False)

# Initialize the model and optimize
mode = 'max'
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                         mode=mode, factor=0.5, patience=2)

In [13]:
num_epochs = 30

train_loss_tracker = torch.zeros(num_epochs).to(device)
test_loss_tracker = torch.zeros(num_epochs).to(device)

tic = time.time()
te = tic

# Train the model
for epoch in range(num_epochs):
    model.train()
    train_acc = torch.tensor(0).type(torch.float).to(device)
    for i, (train_data, train_labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(train_data[0].to(device), 
                        train_data[1].to(device)).to(device)
        probs, preds = torch.max(outputs, dim=1)
        train_labels = train_labels.to(device)
        if loss_fn_name == 'Cross':
            loss = loss_fn(outputs.max(dim=1)[0], 
                           train_labels.argmax().type(torch.long))
        else:
            loss = loss_fn(outputs, train_labels)
        train_loss_tracker[epoch] += loss / (int(labs.shape[0]*0.8))
        train_acc += sum(preds == train_labels.argmax()) / \
                        (int(labs.shape[0]*0.8))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
    model.eval()
    acc = torch.tensor(0).type(torch.float).to(device)
    with torch.no_grad():
        for i, (test_data, test_labels) in enumerate(test_loader):
            outputs = model(test_data[0].to(device), 
                            test_data[1].to(device)).to(device)
            ps, preds = torch.max(outputs, dim=1)
            test_labels = test_labels.to(device)
            if loss_fn_name == 'Cross':
                test_loss = loss_fn(outputs.max(dim=1)[0], 
                                    test_labels.argmax().type(torch.long))
            else:
                test_loss = loss_fn(outputs, test_labels)
            test_loss_tracker[epoch] += test_loss / \
                    ((labs.shape[0]-int(labs.shape[0]*0.8)))
            acc += sum(preds == test_labels.argmax()) / \
                   (labs.shape[0]-int(labs.shape[0]*0.8))
    
    scheduler.step(acc)

    print(f'Epoch {epoch+1}:  Test Acc: {round(acc.item()*100, 3)}', end='')
    print(f'\tTrain Loss: {round(train_loss_tracker[epoch].item(), 3)}', end='')
    print(f'\tTest Loss: {round(test_loss_tracker[epoch].item(), 3)}', end='')
    print(f'\tTrain Acc: {round(train_acc.item()*100, 3)}', end='')
    print(f'\tRuntime: {round(time.time()-te, 0)}')
    te = time.time()
    
toc = time.time()
print(f'Training Runtime: {toc-tic}s')

Epoch 1:  Test Acc: 52.308	Train Loss: 1.001	Test Loss: 0.946	Train Acc: 50.169	Runtime: 1132.0


KeyboardInterrupt: 